# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
  
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>

## <center>Projeto Final</center>

**Aluno**: Eduardo Garcia do Nascimento

**RA/CPF**: 22008732800


---
### Análise exploratória

Nesta seção, deve ser feita a leitura da base de dados e todas as análises necessárias para entendê-la melhor, tais como:
* Significado de cada atributo
* Medidas descritivas
* Gráficos

In [ ]:
# -*- coding: utf-8 -*-

# Caminho dos arquivos
FILES_DIRECTORY = "data"

import numpy as np 
import pandas as pd 
import os
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from scripts import utils

if __name__ == '__main__':
                       
    # importa o arquivo e guarda em um dataframe do Pandas
    set1_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set1.csv'), sep=',', low_memory=False)
    set2_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set2.csv'), sep=',', low_memory=False) 
    set3_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set3.csv'), sep=',', low_memory=False)
    
    # Renomeia colunas concatenando o setX antes de fazer o merge para identificá-las posteriormente
    cols = set1_dataset.columns
    for col in cols:
        set1_dataset = set1_dataset.rename(columns={col:'set1_'+col})
        
    cols = set2_dataset.columns
    for col in cols:
        set2_dataset = set2_dataset.rename(columns={col:'set2_'+col})
    
    cols = set3_dataset.columns
    for col in cols:
        set3_dataset = set3_dataset.rename(columns={col:'set3_'+col})

    # Concatena os datasets em somente um dataset único
    frames = [ set1_dataset, set2_dataset, set3_dataset ]
    input_dataset = pd.concat(frames, axis=1)
    
    print('A base de dados inicial combinada tem %d amostras com %d atributos' % (input_dataset.shape[0],
                                                                                 input_dataset.shape[1]))
    
    # Remove os atributos que são constantes e não oferecem nenhum valor aos algoritmos de classificação
    variance_mask = VarianceThreshold().fit(input_dataset).get_support()
    input_dataset = input_dataset.iloc[:,variance_mask]
    print('Atributos removidos por baixa variância: %d' % np.sum(~variance_mask))
    
    # Tratamento de outliers e entradas nulas
    df = input_dataset.copy()
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    mask = (df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))
    print("Número de outliers substituídos por valores nulos:", np.sum(np.sum(mask)))
    #df[mask] = np.nan
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    input_dataset.loc[:,:] = imp_mean.fit_transform(input_dataset)
    
    # Normalização dos dados entre 0 e 1
    input_dataset.loc[:,:] = MinMaxScaler().fit_transform(input_dataset)
    
    # Adiciona as classes junto ao dataset de atributos
    train_dataset = pd.read_csv(os.path.join(FILES_DIRECTORY, 'train.csv'), sep=',')
    input_dataset['classe'] = np.nan
    input_dataset.loc[train_dataset['Id'].values,'classe'] = train_dataset['Class'].values
    
    backup_dataset = input_dataset.copy()
    
    mask = ((input_dataset['classe'] == -1) | (input_dataset['classe'] == 1))
    display(input_dataset.loc[mask].head(10))
    
    cols = list(input_dataset.columns)
    cols.remove('classe')
        
    # Seleciona os melhores atritubos para treinametno do algoritmo de classificação
    print('Selecionando melhores features....................................')
    selector = SelectKBest(f_classif, k=6).fit(
        input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1), cols].values,
        input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1),'classe'].values)
    mask = selector.get_support()
    mask = np.append(mask, True)
    input_dataset = input_dataset.iloc[:,mask]
    
    # matriz de gráficos scatter 
    sns.pairplot(input_dataset, hue='classe', height=3.5);
    plt.show()
    
    # matrizes de covariancia e correlação
    df_covariance = input_dataset.iloc[:,:-1].cov()
    df_correlation = input_dataset.iloc[:,:-1].corr()
    
    
    # cria um mapa de cores dos valoes da covariancia
    sns.heatmap(df_covariance, annot=True, xticklabels=df_correlation.columns, yticklabels=df_correlation.columns)
    plt.title('Covariância')
    plt.show()

    # cria um mapa de cores dos valoes da correlação
    sns.heatmap(df_correlation, annot=True, xticklabels=df_correlation.columns, yticklabels=df_correlation.columns)
    plt.title('Correlação')
    plt.show()
    
    display(input_dataset.info())
    
    print('Dados concatenados produzindo um total de %d atributos' % 
            input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)].shape[1])
    
    print('O número de amostras com classificação válida é: %d' % 
            input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)].shape[0])

    print('Dados de treinamento carregados com sucesso!')

    test_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'test.csv'), sep=',')
    K = input_dataset.loc[test_dataset.iloc[:,:].values.T[0]]
    K = K.drop('classe', axis=1).values
    
    print('Análise e visualização dos dados:')
    y = input_dataset.classe
    ax = sns.countplot(y, label="Contagem")
    N,U,P = y.value_counts()
    print('Número de posts comuns:', N)
    print('Número de posts não reconhecidos:', U)
    print('Número de posts phishing', P)
    plt.show()
    
    display(input_dataset.describe())
    
    cl = input_dataset.columns
    cl = cl.drop('classe')
    
 
    plt.figure(figsize=(20,10))
    data = pd.melt(input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)],
                   id_vars="classe", var_name="features", value_name='value')
    sns.boxplot(x='features', y='value', hue='classe', data=data)
    plt.show()
    
    plt.figure(figsize=(20,10))
    sns.violinplot(x='features', y='value', hue='classe', data=data, split=True, inner="quartile")
    plt.show()
    
    X = input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)]
    X_total = X.drop('classe', axis=1).values
    y_total = input_dataset.loc[((input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)), 'classe'].values
    
    utils.printPCA(X_total,y_total)    
    
    utils.beep(1, 400)

---
### Pré-processamento

Nesta seção, as funções da etapa de pré-processamento dos dados devem ser implementadas e aplicadas (se necessário)

In [ ]:
from sklearn.model_selection import train_test_split
from scripts import utils
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

evaluate_svm_hiperparameters = False # Busca os melhores parâmetros para as máquinas de vetores de suporte
evaluate_rfc_hiperparameters = False # Busca os melhores parâmetros para as florestas aleatórias
evaluate_lrc_hiperparameters = False # Busca os melhores parâmetros para a regressão linear

print('Removendo amostras outliers')
X, y = utils.remove_outliers(X_total, y_total)
utils.printPCA(X,y)

print(X.shape, y.shape)
print('Separando a base em treino e teste')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0, shuffle=True, stratify=y)


utils.printBoxPlot(X)

cv = StratifiedKFold(n_splits=4, random_state=1, shuffle=True)

scores = ['balanced_accuracy', 'f1', 'roc_auc']

if evaluate_svm_hiperparameters == True:
    print('Buscando os melhores parâmetros para as máquinas de vetores de suporte:')
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': ['scale', 1e-3, 1e-4], 'C': [1, 10, 100, 200, 400, 600, 1000]},
                        {'kernel': ['linear'], 'C': [1, 10, 100, 200, 400, 600, 1000]},
                        {'kernel': ['poly'], 'C': [1, 10, 100, 200, 400, 600, 1000], 'degree': [2,3,4,5,6], 'gamma': ['scale', 1e-3, 1e-4]}]

    

    for score in scores:
        clf = GridSearchCV(
                svm.SVC(class_weight='balanced', random_state=1), tuned_parameters, scoring=score, cv=cv
            )
        clf.fit(X, y)

        print('Os melhores parâmetros encontrados para a pontuação %s foram:' % score)
        print(clf.best_params_)

    
if evaluate_rfc_hiperparameters == True:    
    print('Buscando os melhores parâmetros para as florestas aleatórias:')

    tuned_parameters = { 'n_estimators': [100, 200, 500],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'max_depth' : [None, 2,3,4,5,6,7,8],
                         'criterion' :['gini', 'entropy']}

    for score in scores:
        clf = GridSearchCV(
                RandomForestClassifier(class_weight='balanced'), tuned_parameters, scoring=score, cv=cv
            )
        clf.fit(X_train, y_train)

        print('Os melhores parâmetros encontrados para a pontuação %s foi:' % score)
        print(clf.best_params_)

utils.beep(2, 600)



---
### Experimento

Nesta seção, o experimento deve ser conduzido, utilizando os protocolos experimentais ensinados no curso e executando os métodos inteligentes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import  plot_confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.semi_supervised import LabelSpreading
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import resample

submit_by_preference = True
submit_preference_score = 'SVM rbf_ra'

submit_by_score = False
submit_best_score = 'balanced_accuracy'

scoring_list=['roc_auc', 'f1', 'f1_micro', 'f1_macro', 'f1_weighted', 'accuracy', 'balanced_accuracy']

results = pd.DataFrame()

model_list = [
    ['SVM poly', svm.SVC(kernel='poly', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
    ['SVM linear', svm.SVC(kernel='linear', C=100, class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
    ['SVM rbf', svm.SVC(kernel='rbf', C=1000, gamma=0.0001, class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
    #['SVM poly_ra', svm.SVC(kernel='poly', C=1000, degree=6, gamma='scale', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
    ['SVM rbf_ra', svm.SVC(kernel='rbf', C=10, gamma='scale', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
    ['SVM rbf_ba', svm.SVC(kernel='rbf', C=400, gamma='scale', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
    ['SVM rbf_f1', svm.SVC(kernel='rbf', C=600, gamma=0.001, class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
    ['Random Forest', RandomForestClassifier(max_depth=2, class_weight='balanced', random_state=1), 0],
    ['Logistic Regression', LogisticRegression(random_state=1, class_weight='balanced', max_iter=15000), 0],
    ['Multinomial NB', MultinomialNB(), 0],
    ['KNN', KNeighborsClassifier(weights='distance'), 0]
]

# Pré balanceamento dos dados utilizando a técnica de oversampling
X_train_bal, y_train_bal = utils.balance_classes(X_train, y_train)
X_bal = X
y_bal = y
#X_bal, y_bal = utils.balance_classes(X, y)

semi_model = svm.SVC(kernel='poly', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1)
if True:
    # População dos dados não classificados com o melhor classificador encontrado com os resultados mais confiáveis
    X_semi_df = backup_dataset[(backup_dataset.classe!=-1)&(backup_dataset.classe!=1)&(backup_dataset.classe!=0)]
    X_semi_df = X_semi_df.iloc[:,mask]
    model = semi_model
    clf = model.fit(X_train_bal, y_train_bal)
    y_semi = clf.predict(X_semi_df.drop('classe', axis=1).values)
    y_probas = clf.predict_proba(X_semi_df.drop('classe', axis=1).values)
    proba_mask = (y_probas[:,0] < 0.2) | (y_probas[:,0] > 0.8)
    y_semi = y_semi[proba_mask]
    X_semi_df = X_semi_df[proba_mask]
    X_semi = X_semi_df.drop('classe', axis=1).values

    print('Número de amostras acrescentadas ao dataset de treinamento:', X_semi.shape[0], 
                                                                          np.sum(y_semi==-1), np.sum(y_semi==1))
    X_train_bal = np.concatenate([X_train_bal, X_semi], axis=0)
    y_train_bal = np.concatenate([y_train_bal, y_semi], axis=0)
    X_bal = np.concatenate([X_bal, X_semi], axis=0)
    y_bal = np.concatenate([y_bal, y_semi], axis=0)

if True:
    X_train_bal, y_train_bal = utils.balance_classes(X_train_bal, y_train_bal)
    X_bal, y_bal = utils.balance_classes(X_bal, y_bal)

    # População dos dados não classificados com o melhor classificador encontrado com os resultados mais confiáveis
    X_semi_df = backup_dataset[(backup_dataset.classe!=-1)&(backup_dataset.classe!=1)&(backup_dataset.classe!=0)]
    X_semi_df = X_semi_df[~proba_mask]
    X_semi_df = X_semi_df.iloc[:,mask]
    model = semi_model
    clf = model.fit(X_train_bal, y_train_bal)
    y_semi = clf.predict(X_semi_df.drop('classe', axis=1).values)
    y_probas = clf.predict_proba(X_semi_df.drop('classe', axis=1).values)
    proba_mask = (y_probas[:,0] < 0.2) | (y_probas[:,0] > 0.8)
    y_semi = y_semi[proba_mask]
    X_semi_df = X_semi_df[proba_mask]
    X_semi = X_semi_df.drop('classe', axis=1).values
    print('Número de amostras acrescentadas ao dataset de treinamento:', X_semi.shape[0], 
                                                                          np.sum(y_semi==-1), np.sum(y_semi==1))
    X_train_bal = np.concatenate([X_train_bal, X_semi], axis=0)
    y_train_bal = np.concatenate([y_train_bal, y_semi], axis=0)
    X_bal = np.concatenate([X_bal, X_semi], axis=0)
    y_bal = np.concatenate([y_bal, y_semi], axis=0)

#X_train_bal, y_train_bal = utils.balance_classes(X_train_bal, y_train_bal)
#X_bal, y_bal = utils.balance_classes(X_bal, y_bal)

# Configura K folds estratificados, ou seja, mantendo as mesmas proporções entre classes
cv = StratifiedKFold(n_splits=4, random_state=1, shuffle=True)


for model in model_list:
    print(model[0] + '------------------------------------------------------------------------------------')
    y_pred = utils.evaluate_model(model[1], X_train_bal, y_train_bal, X_test, y_test)
    
    for scoring in scoring_list:
        scores = cross_val_score(model[1], X_bal, y_bal, scoring=scoring, cv=cv, n_jobs=-1)
        print('Cross ' + scoring + ': %.3f (%.3f)------------------' % (np.mean(scores), np.std(scores)))
        results.loc[model[0], scoring] = np.mean(scores)
        if scoring == submit_best_score:
            model[2] = np.mean(scores)
    results.loc[model[0], 'TN'] = confusion_matrix(y_test, y_pred)[0][0]
    results.loc[model[0], 'TP'] = confusion_matrix(y_test, y_pred)[1][1]


if submit_by_score == True:
    best_score = 0
    best_model = 0
    best_model_name = 'None'
    for model in model_list:
        if model[2]>best_score:
            best_score = model[2]
            best_model = model[1]
            best_model_name = model[0]

    print('O modelo que obteve melhores resultados foi: ' + 
      best_model_name + ' com ' + 
      submit_best_score + '=' + str(best_score))
    
if submit_by_preference == True:
    for model in model_list:
        if model[0]==submit_preference_score:
            best_score = model[2]
            best_model = model[1]
            best_model_name = model[0]

    print('O modelo que obteve melhores resultados foi: ' + 
      best_model_name + ' com ' + 
      submit_preference_score + '=' + str(best_score))
    
    
    
    
print('Printing model to submission.csv ###################################################################')
clf= best_model.fit(X_bal, y_bal)
y_pred_submission = clf.predict_proba(K)[:,1]
result = np.zeros((K.shape[0],2))
for i in range(K.shape[0]):
    result[i][0] = test_dataset.iloc[:,:].values.T[0][i]
    result[i][1] = y_pred_submission[i]
resultdf = pd.DataFrame(data=result, columns=["Id", "Predicted"])
resultdf['Id'] = resultdf['Id'].astype(int)
resultdf['Predicted'] = resultdf['Predicted'].round(decimals=5)
resultdf.to_csv('submission.csv', index=False, float_format='%.5f')
print('####################################################################################################')

display(results)
    
utils.beep(3, 800)



---
### Análise dos Resultados

Nesta seção, os resultados devem ser exibidos e comparados, através de tabelas e gráficos

asdfqwerqwrqwerqwer
qeqwerqwer
